**<h1>Implementación del modelo final</h1>**

In [10]:
# Importamos las librerías correspondientes
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

# Cargar los datos en un DataFrame
comp_df = pd.read_csv("/comp_st23ns1.txt.bz2", compression="bz2", index_col=0)
disp_df = pd.read_csv("/disp_st23ns1.txt.bz2", compression="bz2",index_col=0)

# Dividir los datos en 10 años para entrenamiento
numero_filas_train = 365 * 10
train = disp_df.iloc[:numero_filas_train, :]

# Las pruebas se hacen en los datos de competición
test = comp_df.iloc[:]

# Dividir los datos de entrenamiento en las variables (X) y la salida (y)
# Todas las columnas excepto la última
X_train = train.iloc[:, :-1]
# Última columna
y_train = train.iloc[:, -1]
# Todas las columnas ya que los datos de competición no tienen salida
X_test = test

scaler = StandardScaler()
svr = SVR()

pipe_regr = Pipeline([
    ('scale', scaler),
    ('SVM', svr)])

# Definición del espacio de búsqueda de hiperparámetros
param_grid = {'SVM__C': [0.1, 1, 10, 100, 1000, 10000, 100000, 1000000,10000000, 100000000],
              'SVM__gamma': [0.01, 0.1, 1]}

inner = KFold(n_splits=3, shuffle=True, random_state=42)

# Definición de un proceso de 2 pasos que ajusta automáticamente 2 hiperparámetros
modelo_final = GridSearchCV(pipe_regr, 
                        param_grid,
                        scoring='neg_mean_squared_error',
                        cv=inner, 
                        n_jobs=4, verbose=1)

# Se entrena el proceso de ajuste automático
np.random.seed(42)
modelo_final.fit(X=X_train, y=y_train.values.ravel())

# 'modelo_final' contiene el modelo con los mejores hiperparámetros encontrados por gridsearch
# y entrenado en el conjunto completo de 'X_train'

# Guardamos el modelo final en un archivo pickle
with open('modelo_final.pkl', 'wb') as file:
    pickle.dump(modelo_final, file)

# Cargamos el modelo desde el archivo pickle
with open('modelo_final.pkl', 'rb') as file:
    modelo_final = pickle.load(file)

# Se obtienen las predicciones para el conjunto de prueba
y_pred = modelo_final.predict(X_test)

# Finalmente, se guardan las predicciones en un archivo CSV
predicciones_df = pd.DataFrame({'Predicciones': y_pred})
predicciones_df.to_csv('predicciones.csv', index=False)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
